# Surebets

Situation where you can profit regardless of a match outcome based on different bookmakers providing different odds

If (1/odds1) + (1/odds2) <1.0, you've got a sure bet


| Bookie | Over     | Under     |
|--------|----------|-----------|
|   1    |   **2**  |   1.9     |
|   2    |  1.8     |  **2.1**  |


With $100 to bet. $51.21 on the over with bookie 1, $48.78 on the under with bookie 2

**Scenario 1 - Over**
You win on Bookmaker 1 but lose on Bookmaker 2
`Profit 1= 51.21*2 — 51.21 — 48.78 = 2.43`

**Scenario 2 - Under**
You win on Bookmaker 2 but lose on Bookmaker 1
`Profit 2 = 48.78*2.1 — 51.21 — 48.78= 2.44`

In [4]:
from selenium import webdriver
import pandas as pd
import time

ModuleNotFoundError: No module named 'selenium'

In [1]:
def find_surebet(odds1, odds2):
    sum_odds = (1/odds1) + (1/odds2)
    if sum_odds<1:
        message = 'Surebet Found!'
    else:
        message = 'Keep scraping'
    return message